# What to do when you get an error

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
# Install required libraries for Transformers, datasets, evaluation, and git-lfs for large file storage
!uv pip install datasets evaluate transformers[sentencepiece]
!apt install git-lfs

You will need to setup git, adapt your email and name in the following cell.

In [ ]:
# Configure Git with your credentials - required for pushing models to the Hub
# Replace with your actual email and name
!git config --global user.email "you@example.com"
!git config --global user.name "Your Name"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [ ]:
# Login to Hugging Face Hub - you'll need your access token
# This allows you to download private models and upload your own models
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Duplicate login cell (can be removed)
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Function to copy a model repository template and create a new repository
# This demonstrates how to clone an existing model and create your own copy
from distutils.dir_util import copy_tree
from huggingface_hub import Repository, snapshot_download, create_repo, get_full_repo_name


def copy_repository_template():
    # Clone the repo and extract the local path
    # Using a specific commit hash ensures reproducibility
    template_repo_id = "lewtun/distilbert-base-uncased-finetuned-squad-d5716d28"
    commit_hash = "be3eaffc28669d7932492681cd5f3e8905e358b4"
    template_repo_dir = snapshot_download(template_repo_id, revision=commit_hash)
    
    # Create an empty repo on the Hub with the same name
    model_name = template_repo_id.split("/")[1]
    create_repo(model_name, exist_ok=True)
    
    # Clone the empty repo locally for modification
    new_repo_id = get_full_repo_name(model_name)
    new_repo_dir = model_name
    repo = Repository(local_dir=new_repo_dir, clone_from=new_repo_id)
    
    # Copy all files from template to new repo
    copy_tree(template_repo_dir, new_repo_dir)
    
    # Push to Hub - uploads all copied files
    repo.push_to_hub()

In [ ]:
# Attempt to load a question-answering pipeline with the copied model
# This will fail because the model is missing a config.json file
from transformers import pipeline

model_checkpoint = get_full_repo_name("distillbert-base-uncased-finetuned-squad-d5716d28")
reader = pipeline("question-answering", model=model_checkpoint)

In [ ]:
# Second attempt - still fails for the same reason (missing config.json)
# The error message helps us understand what's missing
model_checkpoint = get_full_repo_name("distilbert-base-uncased-finetuned-squad-d5716d28")
reader = pipeline("question-answering", model=model_checkpoint)

In [ ]:
# Investigate what files are actually in the repository
# This helps us confirm that config.json is missing
from huggingface_hub import list_repo_files

list_repo_files(repo_id=model_checkpoint)

In [ ]:
# Create a config from the base model that this model was fine-tuned from
# The config.json file contains essential model architecture information
from transformers import AutoConfig

pretrained_checkpoint = "distilbert-base-uncased"
config = AutoConfig.from_pretrained(pretrained_checkpoint)

In [ ]:
# Push the config to the repository to fix the missing config.json issue
# This adds the essential configuration file needed for model loading
config.push_to_hub(model_checkpoint, commit_message="Add config.json")

In [ ]:
# Now we can successfully load the question-answering pipeline
# Using revision="main" ensures we get the latest version with config.json
reader = pipeline("question-answering", model=model_checkpoint, revision="main")

# Test the pipeline with sample text about extractive question answering
context = r"""
Extractive Question Answering is the task of extracting an answer from a text
given a question. An example of a question answering dataset is the SQuAD
dataset, which is entirely based on that task. If you would like to fine-tune a
model on a SQuAD task, you may leverage the
examples/pytorch/question-answering/run_squad.py script.

🤗 Transformers is interoperable with the PyTorch, TensorFlow, and JAX
frameworks, so you can use your favourite tools for a wide variety of tasks!
"""

question = "What is extractive question answering?"
reader(question=question, context=context)

In [ ]:
# Extract the tokenizer and model from the pipeline for manual debugging
# This gives us direct access to the underlying components
tokenizer = reader.tokenizer
model = reader.model

In [ ]:
# Prepare a new question to test manual processing
question = "Which frameworks can I use?"

In [ ]:
# Manual question answering processing - demonstrates what happens inside the pipeline
# This will fail because inputs are not properly converted to tensors
import torch

# Tokenize the question and context together
inputs = tokenizer(question, context, add_special_tokens=True)
input_ids = inputs["input_ids"][0]  # This line will cause an error

# Forward pass through the model
outputs = model(**inputs)
answer_start_scores = outputs.start_logits
answer_end_scores = outputs.end_logits

# Find the most likely start and end positions
answer_start = torch.argmax(answer_start_scores)
answer_end = torch.argmax(answer_end_scores) + 1

# Convert token IDs back to text
answer = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
)
print(f"Question: {question}")
print(f"Answer: {answer}")

In [ ]:
# Investigate the structure of tokenizer output
# Check the first 5 tokens and data type to understand the issue
inputs["input_ids"][:5]

In [ ]:
# Check the data type of input_ids
# The issue is that input_ids is a list, not a tensor
type(inputs["input_ids"])